In [55]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn import metrics

from sklearn.feature_selection import RFE


## Метод рекурсивного исключения признаков

In [56]:
# создаем функцию для вывода метрик
def print_metrics(y_train, y_train_predict, y_test, y_test_predict):
    print('Train R^2: {:.3f}'.format(metrics.r2_score(y_train, y_train_predict)))
    print('Train MAE: {:.3f}'.format(metrics.mean_absolute_error(y_train, y_train_predict)))
    print('Train MAPE: {:.3f}'.format(metrics.mean_absolute_percentage_error(y_train, y_train_predict)*100))
    print('\n')
    print('Test R^2: {:.3f}'.format(metrics.r2_score(y_test, y_test_predict)))
    print('Test MAE: {:.3f}'.format(metrics.mean_absolute_error(y_test, y_test_predict)))
    print('Train MAPE: {:.3f}'.format(metrics.mean_absolute_percentage_error(y_test, y_test_predict)*100))

In [57]:
data = pd.read_excel('data/data_ford_price.xlsx') 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


In [58]:
#предобработка данных
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']               #выделяем целевой признак
x = data.drop(columns='price')  #удаляем целевой признак из датасета
# делим датасетна тренировочную и тестовую выборку

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [59]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)# обучем

In [60]:
col_name = selector.get_feature_names_out()
X_train = X_train[col_name]
print(X_train.head(1))
X_test = X_test[col_name]
print(X_test.head(1))

      year  cylinders        lat
4641  2011          6  31.316064
      year  cylinders       lat
3556  2009          8  44.87313


In [61]:
l_regres = LinearRegression() #задаем класс линейной регресии
l_regres.fit(X_train,y_train)# обучем

y_train_pred = l_regres.predict(X_train)
y_test_pred = l_regres.predict(X_test)

print_metrics(y_train, y_train_pred, y_test, y_test_pred)

Train R^2: 0.480
Train MAE: 5283.260
Train MAPE: 11445.391


Test R^2: 0.576
Test MAE: 5096.570
Train MAPE: 16294.887


##  МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

In [62]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [63]:

from sklearn.feature_selection import SelectKBest, f_regression
selector = SelectKBest(f_regression, k=3) #задаем класс 
selector.fit(X_train, y_train)# обучем

 
col_name = selector.get_feature_names_out()

X_train = X_train[col_name]
print(X_train.head(1))
X_test = X_test[col_name]
print(X_test.head(1))

      year  cylinders  odometer
4641  2011          6    122882
      year  cylinders  odometer
3556  2009          8    154597


In [64]:
l_regres = LinearRegression() #задаем класс линейной регресии
l_regres.fit(X_train,y_train)

y_train_pred = l_regres.predict(X_train)
y_test_pred = l_regres.predict(X_test)

print_metrics(y_train, y_train_pred, y_test, y_test_pred)

Train R^2: 0.511
Train MAE: 4896.205
Train MAPE: 11902.047


Test R^2: 0.602
Test MAE: 4708.946
Train MAPE: 17212.254


Вывод:были выбраны по три признака 2 методами RFE и на основе фильтров, после обучения модели  линейной регресии и вывода метрик видно что метод вцбора признака справился лутче так кк имеет возможность подбирать нужные нам  стат.меру